In [4]:
!pip install malaya

     |████████████████████████████████| 2.2MB 5.3MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 245kB 26.9MB/s 
     |████████████████████████████████| 286kB 31.9MB/s 
     |████████████████████████████████| 1.2MB 27.5MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=248116aa32cd9c321bc33fbf203c10bc3feeda989c6a462987b7aa25cde75d12
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ftfy


In [2]:
import malaya
import os 
import collections
import pickle
import string
import re

/usr/local/lib/python3.7/dist-packages/malaya/function/__init__.py:32: UserWarning: Cannot import beam_search_ops not available for Tensorflow 2, `deep_model` for stemmer will not available to use.
  'Cannot import beam_search_ops not available for Tensorflow 2, `deep_model` for stemmer will not available to use.'


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import data

In [3]:
os.chdir('/content/drive/MyDrive/data/pickle')

In [4]:
# Get a list of all hansards
hansards_names = os.listdir()
print(hansards_names)

['hansard_p12', 'hansard_p13', 'hansard_p14', 'hansard_p11', 'hansard_p10', 'hansard_p09', 'hansard_p08']


In [5]:
# Read data into a dict by year 
hansards_dict_by_year = {}

for hansard in hansards_names:
    data = pickle.load(open(hansard, 'rb'))
    for doc in data:
        key = re.search('\d{8}', doc[0].replace(' ', '')).group(0)[-4:]
        content = doc[1]
        if key in hansards_dict_by_year:
            hansards_dict_by_year[key].append(content)
        else:
            hansards_dict_by_year[key] = [content]
            
print(hansards_dict_by_year.keys())

dict_keys(['2010', '2008', '2009', '2012', '2011', '2014', '2015', '2013', '2017', '2016', '2018', '2019', '2020', '2006', '2004', '2007', '2005', '2002', '2000', '2003', '2001', '1999', '1995', '1996', '1997', '1998', '1991', '1993', '1994', '1992', '1990'])


In [6]:
hansards_dict_clean = {}

for key, docs in hansards_dict_by_year.items():
  for doc in docs:
    if key in hansards_dict_clean:
      hansards_dict_clean[key].append(doc.replace('\n', ' ').replace('  ', ' '))
    else:
      hansards_dict_clean[key] = [doc.replace('\n', ' ').replace('  ', ' ')]

In [11]:
od = collections.OrderedDict(sorted(hansards_dict_clean.items()))

for key, value in od.items():
    print(key, len(value))

1990 15
1991 96
1992 65
1993 72
1994 67
1995 66
1996 87
1997 85
1998 71
1999 49
2000 83
2001 77
2002 61
2003 67
2004 68
2005 78
2006 91
2007 85
2008 79
2009 71
2010 84
2011 63
2012 70
2013 51
2014 57
2015 61
2016 55
2017 57
2018 70
2019 68
2020 55


# Entity Recognition

In [19]:
string = '''Saya pun tidak faham dia cakap. Tetapi saya Hokkien. Sekarang ini saya tengok pelancong daripada Arab datang dia bawa anak dia semua, takkan dia mahu bawa anak dia yang kecil-kecil masuk disko. Pelancong pun kalau hendakkan hiburan di negara mereka pun hiburan mereka lebih lagi. Macam mana hiburan
dia hendak pun ada, naik atas meja, naik atas kerusi pun ada hiburan depa. Di
Malaysia ini pelancong mai, maksud saya ada pusat hiburan yang dia boleh
pergi ke pub, dia boleh pergi di karaoke, hiburanlah. Bukan maksud saya
hiburan keterlaluan sampai tidak ada batasan dan malu kita Tuan Yang di-
Pertua kalau anak-anak muda, anak-anak Malaysia kita, anak-anak perempuan
kita ini tahu-tahu pi jadi pelacur, jadi GRO, pelancongpelancong bawa budakbudak
Malaysia berjalan. Malu Tuan Yang di-Pertua. Kita tengok malu. Anakanak
kita mahu orang Mat Saleh atau orang Negro, mai pegang berjalan. Kita
tengok, Seputeh, tak malu kah? Sekarang you jalan sama orang Negro, you ada
malu kah?... [Ketawa] [Dewan riuh dengan ketawa] Kita pun takut ho...!'''

In [15]:
model = malaya.entity.transformer_ontonotes5(model = 'tiny-albert')

downloading frozen model to /root/Malaya/entity-ontonotes5/tiny-albert/model.pb


23.0MB [00:00, 30.5MB/s]                          


downloading frozen vocab to /root/Malaya/entity-ontonotes5/tiny-albert/sp10m.cased.v10.vocab


  0%|          | 0.00/0.54 [00:00<?, ?MB/s]/usr/local/lib/python3.7/dist-packages/tqdm/std.py:484: TqdmWarning: clamping frac to range [0, 1]
  charset=Bar.ASCII if ascii is True else ascii or Bar.UTF)
184%|██████████| 1.00/0.54 [00:00<00:00, 5.43MB/s]


downloading frozen tokenizer to /root/Malaya/entity-ontonotes5/tiny-albert/sp10m.cased.v10.model


135%|██████████| 1.00/0.74 [00:00<00:00, 4.31MB/s]


downloading frozen setting to /root/Malaya/entity-ontonotes5/tiny-albert/entities-ontonotes5.json


1.00MB [00:00, 189MB/s]                    


In [ ]:
model.predict(string)
#model.predict(hansards_dict_clean['2020'][0][5000:6000])

# POS tagging

In [7]:
model = malaya.pos.transformer(model = 'tiny-albert')

In [ ]:
adj_dict = {}

for year, docs in hansards_dict_clean.items():
  adj_year = []
  print(year)
  for doc in docs:
    total = len(doc)
    for i in range(0, total, 1000): # predict using a 1000-words interval due to model limits 
      try:
        adj = [i[i[1]=='ADJ'] for i in model.predict(doc[i:i+1000])]
        adj_year.extend(adj)
      except:
        continue
  
  adj_dict[year] = adj_year

1991
1993


In [1]:
print(adj_dict.keys())

NameError: ignored

In [50]:
# for year, docs in hansards_dict_clean.items():
#   adj_year = []
#   print(year)
#   for doc in docs:
#     total = len(doc)

adj_set = []
year = '2001'

for doc in hansards_dict_clean[year]:
  total = len(doc)
  for i in range(0, total, 1000): # predict using a 1000-words interval due to model limits 
    try:
      adj = [i[0] for i in model.predict(doc[i:i+1000]) if i[1] == 'ADJ']
      adj_set.extend(adj)
    except:
      continue

In [51]:
adj_set = set(adj_set)
len(adj_set)

14783

In [52]:
# Save models 
pickle.dump(adj_set, open('../adj/adj_' + year, 'wb'))